In [1]:
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score

from BiHDM import BiHDMClassifier

In [2]:
# Define 64 EEG channels using 10-20 standard
ch_names = ['Fp1', 'AF7', 'AF3', 'F1', 'F3', 'F5', 'F7', 'FT7', 'FC5', 'FC3', 'FC1', 
            'C1', 'C3', 'C5', 'T7', 'TP7', 'CP5', 'CP3', 'CP1', 'P1', 'P3', 'P5', 
            'P7', 'P9', 'PO7', 'PO3', 'O1', 'Iz', 'Oz', 'POz', 'Pz', 'CPz', 'Fpz', 
            'Fp2', 'AF8', 'AF4', 'AFz', 'Fz', 'F2', 'F4', 'F6', 'F8', 'FT8', 'FC6', 
            'FC4', 'FC2', 'FCz', 'Cz', 'C2', 'C4', 'C6', 'T8', 'TP8', 'CP6', 'CP4', 
            'CP2', 'P2', 'P4', 'P6', 'P8', 'P10', 'PO8', 'PO4', 'O2']

lh_chs = ['Fp1', 'AF7', 'AF3', 'F7', 'F5', 'F3', 'F1', 'FT7', 'FC5', 'FC3', 'FC1', 
          'T7', 'C5', 'C3', 'C1', 'TP7', 'CP5', 'CP3', 'CP1', 'P7', 'P5', 'P3', 'P1', 
          'PO7', 'PO3', 'O1']
rh_chs = ['Fp2', 'AF8', 'AF4', 'F8', 'F6', 'F4', 'F2', 'FT8', 'FC6', 'FC4', 'FC2', 
          'T8', 'C6', 'C4', 'C2','TP8', 'CP6', 'CP4', 'CP2', 'P8', 'P6', 'P4', 'P2', 
          'PO8', 'PO4', 'O2']
lv_chs = ['Fp1', 'AF7', 'F7', 'FT7', 'T7', 'TP7', 'P7', 'PO7', 'AF3', 'F5', 'FC5', 
          'C5', 'CP5', 'P5', 'O1', 'F3', 'FC3', 'C3', 'CP3', 'P3', 'PO3', 'F1', 'FC1', 
          'C1', 'CP1', 'P1']
rv_chs = ['Fp2', 'AF8', 'F8', 'FT8', 'T8', 'TP8', 'P8', 'PO8', 'AF4', 'F6', 'FC6', 
          'C6', 'CP6', 'P6', 'O2', 'F4', 'FC4', 'C4', 'CP4', 'P4', 'PO4', 'F2', 'FC2', 
          'C2', 'CP2', 'P2']

In [3]:
# Generate some data for classification
X = np.ones((1000, 64, 5)) # 1000 samples x 64 channels x 5 bands per channel (delta, theta, alpha, beta, gamma)
y = np.repeat([0,1], 500)

# Let's simulate a frontal alpha-asymmetry for the classifier to learn from
left_frontal_chs = ['Fp1', 'AF3', 'AF7', 'F1', 'F3', 'F5', 'FC3', 'FC1']
X[:500,np.isin(ch_names, left_frontal_chs),2] -= 1

# And let's add some gaussian noise
rng = np.random.default_rng(42)
X += rng.normal(scale=0.5, size=X.shape)

# Reshape X to meet sklearn standard
X = X.reshape(1000, -1)

In [4]:
clf = BiHDMClassifier(ch_names, lh_chs, rh_chs, lv_chs, rv_chs, 
                    d_stream=32, d_pair=32, d_global=32, d_out=16, 
                    k=6, a=0.01, pairwise_operation='subtraction', 
                    rnn_stream_kwargs={}, rnn_global_kwargs={}, 
                    loss='NLLLoss', optimizer='SGD', lr=0.003,
                    epochs=8, batch_size=200, loss_kwargs={}, 
                    optimizer_kwargs=dict(momentum=0.9, weight_decay=0.95),
                    random_state=42, use_gpu=True, verbose=False)

pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('BiHDM', clf)
])

scores = cross_val_score(clf, X, y)
print(np.mean(scores))
print(scores)

0.783
[0.745 0.76  0.775 0.805 0.83 ]
